In [63]:
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.keras.layers import Lambda, Input, Embedding, Dense, TimeDistributed, Dropout, BatchNormalization, Add, SpatialDropout1D
from tensorflow.keras.layers import CuDNNLSTM as LSTM

vocab_size=30000
inp=Input(batch_shape=(1,1,),dtype=tf.int64,name="input_sent")
emb_layer=Embedding(input_dim=vocab_size,output_dim=1500,embeddings_initializer=tf.keras.initializers.Constant(0.01))
emb=emb_layer(inp)
emb_do=SpatialDropout1D(0.5)(emb, training=False)
emb_drop_n=BatchNormalization()(emb_do)
rnn1=LSTM(1500,return_sequences=True,stateful=True)(emb_drop_n)
rnn1_n=BatchNormalization()(rnn1)
rnn2=LSTM(1500,return_sequences=True,stateful=True)(rnn1_n)
rnn2_n=BatchNormalization()(rnn2)
rnn3=LSTM(1500,return_sequences=True,stateful=True)(rnn2_n)
rnn3_n=BatchNormalization()(rnn3)
proj_weights=TimeDistributed(Dense(1500))(Add()([rnn1_n,rnn2_n,rnn3_n]))
proj_weights_gelu=Lambda(lambda x: tf.multiply(x,tf.nn.sigmoid(tf.scalar_mul(1.702,x))))(proj_weights)
dec=Dense(vocab_size,activation="softmax",name="decision")
dec_td=TimeDistributed(dec)(proj_weights_gelu)
mod=tf.keras.Model(inputs=[inp],outputs=[dec_td])


In [64]:
mod.load_weights("models/epoch.2019-03-24-00-00.00020.last.rnnlm")
print("Done")

Done


In [27]:
import data
dp=data.SubwordDataPipeline("models/spiece_vocab.sp")


Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108,

In [28]:
dp.subword_model.IdToPiece(2)

'<s>'

In [29]:
def reset_model(m):
    lstms=[l for l in m.layers if isinstance(l,tf.keras.layers.CuDNNLSTM)]
    for l in lstms:
        l.reset_states()

In [65]:
import numpy as np

#winner=tf.argmax(res,axis=-1)
#print(winner[0][0].numpy())
#dp.subword_model.IdToPiece(int(winner))

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def prewarm(txt,mod,dp):
    reset_model(mod)
    ids=dp.subword_model.EncodeAsIds(txt)
    for i in ids:
        inp=np.asarray([[i]],dtype=np.int64)
        inp_t=tf.convert_to_tensor(inp,tf.int64)
        res=mod(inp_t)
        winner=tf.argmax(res,axis=-1)
        spiece_id=winner[0][0].numpy()
        sw=dp.subword_model.IdToPiece(int(spiece_id))
        print("Prewarm:",sw)
    winner=tf.argmax(res,axis=-1)
    spiece_id=winner[0][0].numpy()
    return int(spiece_id)

#prewarm_text="Meillä on siis pieni ongelma. Tai no oikeastaan aika suuri sellainen. 13-vuotias poikani haluaa nimittäin viikonloppuna mennä kaverinsa ja tämän perheen kanssa Ruotsin risteilylle. Kaveri siis itse ehdotti tätä ja poika haluaa todellakin mennä. Kaikki olisi minullekin täysin OK, jos poikani ei sattuisi olemaan"
prewarm_text="Helsingin alla risteilee jopa yli satavuotisia vesiputkia. Ikä ei kuitenkaan yksin määritä putken kuntoa, ja siksi Helsingissäkään ei pystytä tekemään riskiarvioita vain sen perusteella"
#prewarm_text="Kaupunkien tarpeet ovat Suomessa jääneet lähes tyystin sivuun. Viime vuosina on kuitenkin nähty, että Helsingissä ja sen naapureissa Espoossa ja Vantaalla vuokrat nousevat ja asumisen väljyys"
#prewarm_text="Vantaan terveysasemille hankittiin uusi puhelinpalvelu: soittajat jonottavat jopa 45 minuuttia"
#prewarm_text="Olipa kerran pieni prinsessa"
start=prewarm(prewarm_text,mod,dp)
lst=[]
for _ in range(100):
    inp=np.asarray([[start]],dtype=np.int64)
    inp_t=tf.convert_to_tensor(inp,tf.int64)
    res=mod(inp_t)
    probs=tf.squeeze(res).numpy()
    winner=sample(probs,0.5)
    #winner=np.random.choice(a=np.arange(probs.shape[0]),p=probs)
    #print("argmax:",tf.argmax(res,axis=-1))
    #print("winner:",winner)
    spiece_id=winner
    sw=dp.subword_model.IdToPiece(int(spiece_id))
    lst.append(sw)
    start=spiece_id
print("Final output:")
dp.subword_model.DecodePieces(lst)



Prewarm: ▁yliopiston
Prewarm: ▁on
Prewarm: eile
Prewarm: ▁myös
Prewarm: ▁kaksi
Prewarm: ▁sata
Prewarm: ▁ihmistä
Prewarm: tista
Prewarm: ▁perin
Prewarm: uto
Prewarm: ous
Prewarm: ▁.
Prewarm: M
Prewarm: vää
Prewarm: ▁ole
Prewarm: ▁ole
Prewarm: ▁riitä
Prewarm: itä
Prewarm: ▁,
Prewarm: ▁pituutta
Prewarm: ▁.
Prewarm: ▁vaan
Prewarm: ▁siksi
Prewarm: ▁onkin
Prewarm: kin
Prewarm: ▁ei
Prewarm: ▁ole
Prewarm: tä
Prewarm: ▁tarjoamaan
Prewarm: ▁mitään
Prewarm: al
Prewarm: oita
Prewarm: ▁.
Prewarm: ▁sen
Prewarm: ▁perusteella
Prewarm: ▁,
Final output:


'mitä reittiä on tarkoitus kulkea . Liikenne on myös erittäin suosittu . Vesihuoltolaitoksen toiminta-alueen ulkopuolella on myös useita muita kaupunkeja , kuten esimerkiksi Hakaniemen ja Vesihuoltolaitoksen . Vesihuoltolaitos huolehtii vesihuoltolaitoksen vesihuoltolaitoksen vesihuoltolaitoksen toiminnasta . Vesihuoltolaitos huolehtii vesihuoltolaitoksen vesihuoltolaitoksen toiminta-alueesta . Vesihuoltolaitoksen toiminta-alue on vesihuoltolaitos . Vesihuoltolaitos huolehtii vesihuoltolaitoksen toiminnasta vesihuoltolaitoksen vesihuoltolaitoksen vesihuoltolaitoksen'

In [122]:
def perplexity(text, model, data_pipeline):
    
    from tensorflow.keras.metrics import sparse_categorical_crossentropy
    from tensorflow.keras.backend import exp
    
    reset_model(model)
    text_ids=data_pipeline.subword_model.EncodeAsIds(text)
    predictions=[]
    for subword_id in text_ids[:-1]:
        inp=np.asarray([[subword_id]],dtype=np.int64)
        inp_t=tf.convert_to_tensor(inp,tf.int64)
        res=mod(inp_t)
        probs=tf.squeeze(res).numpy()
        predictions.append(probs)
    
    gold=text_ids[1:]
    inp_p=tf.convert_to_tensor(predictions,tf.float32)
    cross_entropy = sparse_categorical_crossentropy(gold, inp_p).numpy()
    per_word_perplexity = exp(cross_entropy)
    avg_perplexity = exp(np.sum(cross_entropy)/cross_entropy.shape[0]).numpy()
    
    return avg_perplexity, per_word_perplexity

In [129]:
news_document='Euroopan parlamentti on hyväksynyt lopullisesti uuden tekijänoikeusdirektiivin .\
 Direktiivi meni läpi parlamentin täysistunnossa äänin 348–274 .\
 Esitys on ollut poikkeuksellisen kiistanalainen . Sitä ovat puolustaneet oikeuksien haltijat\
 kuten musiikintekijät ja lehtikustantajat . Ehdotettua muutosta ovat vastustaneet etenkin internetin\
 isot alustapalvelut kuten Google ja Facebook . Esitystä vastaan on käyty myös hyvin laajaa\
 kansalaiskampanjointia , jota on motivoinut huoli ilmaisunvapauden rajoituksista .\
 Uuden lainsäädännön 13. artiklan mukaan alustapalveluiden on varmistettava , että niiden palveluissa\
 ei ole tarjolla materiaalia johon niillä ei ole oikeuksia . Kriitikoiden mukaan esitys johtaa käytännössä\
 siihen , että palvelut ryhtyvät varmuuden vuoksi suodattamaan käyttäjiensä niihin lataamaa materiaalia .\
 Esitys rajoittaa myös sitä , millaisia otteita tekijänoikeuksien alaisista lehtiartikkeleista saa esittää\
 internetissä . Tämä " linkkiveroksi " kritisoitu artikla 11 sallii yksittäisten sanojen tai hyvin lyhyiden\
 lauseiden linkkaamisen ilman lisensointia . Jäsenmaat ovat jo tahollaan hyväksyneet esityksen , jonka\
 käsittely EU:ssa päättyi parlamentin äänestykseen . Suomi vastusti sitä .\
 Euroopan parlamentin täysistunto päätti ensin niukalla enemmistöllä , että se ei enää avaa jäsenmaiden\
 kanssa jo sovittua ehdotusta muutosesityksille . Sen jälkeen täysistunto hyväksyi direktiivin jäsenmaiden\
 kanssa neuvotellussa muodossa . Äänestystä edelsi poikkeuksellisen voimakas kampanjointi esityksen puolesta\
 ja sitä vastaan , sillä täysistunnon käsittely nähtiin viimeiseksi tilaisuudeksi vaikuttaa\
 tekijänoikeusdirektiivin sisältöön . Esimerkiksi Saksassa 150 000 ihmistä osoitti viikonvaihteessa mieltään\
 artikloja 11 ja 13 vastaan eri kaupungeissa .'

with open("data/fi_tdt-ud-dev.tok.txt", "rt", encoding="utf-8") as f:
    tdt_devel=f.read()[:10000]

avg_p, word_p = perplexity(news_document, mod, dp)
print("Average perplexity:", avg_p)

avg_p, word_p = perplexity(tdt_devel, mod, dp)
print("Average perplexity:", avg_p)

Average perplexity: 74.60694191699187
Average perplexity: 112.07748615326427
